In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/facad-test-dataset/test_dataset.parquet


In [2]:
!pip install bert-score python-Levenshtein evaluate rouge_score --upgrade nltk

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 35.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 62.3 MB/s eta 0:00:00:00:01
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=0209f10675bb1f7bdf70f2bdd09cbfa12b5cacf55064f4faf43063cfb4c286dd
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beha

In [3]:
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from transformers import AutoProcessor, AutoModelForCausalLM
from bert_score import score as bertscore
from datasets import load_dataset
from tqdm import tqdm
import evaluate 
import Levenshtein

# Load test dataset using the datasets library
data_path = '/kaggle/input/facad-test-dataset/test_dataset.parquet'
test_dataset = load_dataset('parquet', data_files=data_path)['train']
test_images = test_dataset['image']
test_captions = test_dataset['text']

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
# Load evaluation metrics using the evaluate library
bleu = evaluate.load('bleu')
rouge = evaluate.load('rouge')
meteor = evaluate.load('meteor')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [5]:
# Initialize lists to store generated captions
blip_captions, blip_finetuned_captions, git_finetuned_captions = [], [], []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Helper function to generate captions using a given model and processor
def generate_captions(model, processor, test_images):
    captions = []
    model = model.to(device)
    model.eval()  
    with torch.no_grad():
        for image in tqdm(test_images):
            inputs = processor(images=image, return_tensors="pt").to(device)
            pixel_values = inputs.pixel_values
            generated_ids = model.generate(pixel_values=pixel_values, max_length=50)
            caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
            captions.append(caption)
    return captions

In [6]:
# Load models and processors
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")

blip_finetuned_model = BlipForConditionalGeneration.from_pretrained("sagniksengupta/blip-finetuned-facad")
blip_finetuned_processor = BlipProcessor.from_pretrained("sagniksengupta/blip-finetuned-facad")

git_finetuned_model = AutoModelForCausalLM.from_pretrained("sagniksengupta/git-finetuned-facad")
git_finetuned_processor = AutoProcessor.from_pretrained("sagniksengupta/git-finetuned-facad")

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [7]:
# Generate captions from all three models
blip_captions = generate_captions(blip_model, blip_processor, test_images)
blip_finetuned_captions = generate_captions(blip_finetuned_model, blip_finetuned_processor, test_images)
git_finetuned_captions = generate_captions(git_finetuned_model, git_finetuned_processor, test_images)

100%|██████████| 1000/1000 [10:23<00:00,  1.60it/s]


In [20]:
def evaluate_generated_captions(generated_captions, test_captions):
    predictions = generated_captions
    references = [[ref] for ref in test_captions]  

    bleu_scores = bleu.compute(predictions=predictions, references=references)['bleu']
    
    rouge_scores = rouge.compute(predictions=generated_captions, references=test_captions)
    
    P, R, F1 = bertscore(generated_captions, test_captions, lang='en')
    bertscore_f1 = F1.mean().item()
    
    # Levenshtein distance calculation
    levenshtein_distances = []
    for gen_caption, ref_caption in zip(generated_captions, test_captions):
        distance = Levenshtein.distance(gen_caption, ref_caption)
        levenshtein_distances.append(distance)
    
    avg_levenshtein = sum(levenshtein_distances) / len(levenshtein_distances)

    return {
        'bleu': bleu_scores,
        'rouge': rouge_scores['rougeL'],
        'bertscore_f1': bertscore_f1,
        'avg_levenshtein': avg_levenshtein
    }


In [21]:
# Evaluate for each model's captions
blip_metrics = evaluate_generated_captions(blip_captions, test_captions)
blip_finetuned_metrics = evaluate_generated_captions(blip_finetuned_captions, test_captions)
git_finetuned_metrics = evaluate_generated_captions(git_finetuned_captions, test_captions)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaMo

In [22]:
print("BLIP Base Metrics: ", blip_metrics)
print("BLIP Fine-tuned Metrics: ", blip_finetuned_metrics)
print("GIT Fine-tuned Metrics: ", git_finetuned_metrics)

BLIP Base Metrics:  {'bleu': 0.0, 'rouge': 0.1295297393210773, 'bertscore_f1': 0.8311129212379456, 'avg_levenshtein': 97.622}
BLIP Fine-tuned Metrics:  {'bleu': 0.02984166260245386, 'rouge': 0.19304251481728948, 'bertscore_f1': 0.8568896055221558, 'avg_levenshtein': 93.806}
GIT Fine-tuned Metrics:  {'bleu': 0.009336050760516194, 'rouge': 0.1502710551309211, 'bertscore_f1': 0.8445741534233093, 'avg_levenshtein': 92.806}


In [23]:
import pandas as pd

data = [
    {'Model': 'BLIP Base', 'BLEU': blip_metrics['bleu'], 'ROUGE': blip_metrics['rouge'], 'BERTScore F1': blip_metrics['bertscore_f1'], 'Avg. Levenshtein': blip_metrics['avg_levenshtein']},
    {'Model': 'BLIP Fine-tuned', 'BLEU': blip_finetuned_metrics['bleu'], 'ROUGE': blip_finetuned_metrics['rouge'], 'BERTScore F1': blip_finetuned_metrics['bertscore_f1'], 'Avg. Levenshtein': blip_finetuned_metrics['avg_levenshtein']},
    {'Model': 'GIT Fine-tuned', 'BLEU': git_finetuned_metrics['bleu'], 'ROUGE': git_finetuned_metrics['rouge'], 'BERTScore F1': git_finetuned_metrics['bertscore_f1'], 'Avg. Levenshtein': git_finetuned_metrics['avg_levenshtein']}
]

df = pd.DataFrame(data)

df

,Model,BLEU,ROUGE,BERTScore F1,Avg. Levenshtein
0,BLIP Base,0.000000,0.129530,0.831113,97.622
1,BLIP Fine-tuned,0.029842,0.193043,0.856890,93.806
2,GIT Fine-tuned,0.009336,0.150271,0.844574,92.806
